In [1]:
def read_query_as_df(query):
    connection = None
    connection = psycopg2.connect(user="jzztvyjdirgomm", password="974386311e9bf8265574baead65862ee677601c0f8e05bc954785e899d86dfaa", host="ec2-34-247-151-118.eu-west-1.compute.amazonaws.com",port="5432",database="djaki03gmcu3o")
    cursor = connection.cursor()  
    temp = []
    result = None
    cursor.execute(query)
    results = cursor.fetchall()
    colnames = [desc[0] for desc in cursor.description]
    connection.close()
    for result in results:
        result = list(result)
        temp.append(result)
    df = pd.DataFrame(temp, columns=colnames)
    connection.close()
    return df

In [2]:
import psycopg2
import psycopg2.extras
import pandas as pd


query = """
SELECT
    psp.id, psp.created_at, ppmr.material_id, ppmr.amount
FROM production.solar_panel psp
Full JOIN production.panel_material_req ppmr
ON
    psp.panel_type_id = ppmr.panel_type_id
"""
material_req = read_query_as_df(query)
display(material_req)

,id,created_at,material_id,amount
0,84,2022-04-30 06:55:53.842113,28,2.000
1,84,2022-04-30 06:55:53.842113,4,2.000
2,84,2022-04-30 06:55:53.842113,9,3.200
3,84,2022-04-30 06:55:53.842113,3,0.333
4,84,2022-04-30 06:55:53.842113,5,0.300
...,...,...,...,...
3215,335,2022-05-30 13:20:37.540750,6,0.130
3216,335,2022-05-30 13:20:37.540750,7,1.000
3217,335,2022-05-30 13:20:37.540750,8,1.600
3218,335,2022-05-30 13:20:37.540750,1,1.000


In [18]:
import psycopg2
import psycopg2.extras
import pandas as pd


query = """
SELECT
    psp.id, psp.created_at, ppmr.material_id, ppmr.amount
FROM production.solar_panel psp
Full JOIN production.panel_material_req ppmr
ON
    psp.panel_type_id = ppmr.panel_type_id
"""
material_req = read_query_as_df(query)
display(material_req)


material_req.sort_values(by="created_at", inplace = True)
#display(material_req)
panels = material_req['id'].unique()
print(len(panels))
#panels = [114]

for panel in panels[316:322]: #322
    conn = psycopg2.connect(user="jzztvyjdirgomm", password="974386311e9bf8265574baead65862ee677601c0f8e05bc954785e899d86dfaa", host="ec2-34-247-151-118.eu-west-1.compute.amazonaws.com",port="5432",database="djaki03gmcu3o")
    cur = conn.cursor()
    material = material_req[material_req["id"] == panel]
    #display(material)
    print("Panel_id:", panel)
    for index, line in material.iterrows():
        query = """
        SELECT 
        pic.lot_id, 
        sum(pic.quantity),
        min(pl.updated_at) as moved_to_prod
        FROM production.inventory_changelog pic
        JOIN production.lot pl
        ON pic.lot_id = pl.id
        JOIN finance.purchase_order_line ppol
        ON ppol.id = pl.purchase_order_line_id
        WHERE
            bin_id = 2 AND
            ppol.material_id = %s
        GROUP BY
            pic.lot_id
        """%line["material_id"]
        available_lot = read_query_as_df(query)
        
        if available_lot.empty:
            print('Material', line["material_id"],'never added to production')
            break
        if available_lot["sum"].sum() == 0:
            print("No more material left")
            break
        #display(available_lot)
        for index2, row in available_lot.iterrows():
            if round(row["sum"],4) == 0: #if lot empty, go to the next lot
                pass
            elif row["sum"] >= line["amount"]:
                print("Lot has enough material",line["material_id"] ,"left. Stock: ", round(row["sum"],4), ", needed: ", round(line["amount"],4), ", new stock: ", round(row["sum"] - line["amount"],4))
                query = """INSERT INTO production.inventory_changelog (quantity, change_type_id, lot_id, created_at, panel_id) 
                VALUES (%s, 10, %s, %s, %s)
                """
                cur.execute(query, [str(round(-line["amount"],4)), str(row["lot_id"]), line["created_at"], str(panel)])
                line["amount"] = 0
            else:
                print("Multiple lots needed for material:",round(line["material_id"],4) ,". Stock: ", round(row["sum"],4), ", needed: ", round(line["amount"],4), ", new stock: 0")
                #Reduce first lot to 0
                query = """INSERT INTO production.inventory_changelog (quantity, change_type_id, lot_id, created_at, panel_id)
                VALUES (%s, 10, %s, %s, %s)
                """
                cur.execute(query, [str(round(-row["sum"],4)), str(row["lot_id"]), line["created_at"], str(panel)])
                line["amount"] = round(line["amount"]- row["sum"],4) #Reduce lot by remaining stock
                
            if line["amount"] == 0:
                break
        if line["amount"] > 0:
            print('Not enough Material', line["material_id"])
            break
    conn.commit()
    conn.close()

,id,created_at,material_id,amount
0,84,2022-04-30 06:55:53.842113,28,2.000
1,84,2022-04-30 06:55:53.842113,4,2.000
2,84,2022-04-30 06:55:53.842113,9,3.200
3,84,2022-04-30 06:55:53.842113,3,0.333
4,84,2022-04-30 06:55:53.842113,5,0.300
...,...,...,...,...
3215,335,2022-05-30 13:20:37.540750,6,0.130
3216,335,2022-05-30 13:20:37.540750,7,1.000
3217,335,2022-05-30 13:20:37.540750,8,1.600
3218,335,2022-05-30 13:20:37.540750,1,1.000


322
Panel_id: 329
Lot has enough material 7 left. Stock:  63.0 , needed:  1.0 , new stock:  62.0
Lot has enough material 1 left. Stock:  44.0 , needed:  1.0 , new stock:  43.0
Lot has enough material 28 left. Stock:  1838.0 , needed:  2.0 , new stock:  1836.0
Lot has enough material 4 left. Stock:  1838.0 , needed:  2.0 , new stock:  1836.0
Lot has enough material 8 left. Stock:  9.4 , needed:  1.6 , new stock:  7.8
Lot has enough material 9 left. Stock:  18.8 , needed:  3.2 , new stock:  15.6
Lot has enough material 3 left. Stock:  79.772 , needed:  0.333 , new stock:  79.439
Lot has enough material 5 left. Stock:  2.2 , needed:  0.3 , new stock:  1.9
Lot has enough material 6 left. Stock:  2.42 , needed:  0.13 , new stock:  2.29
Multiple lots needed for material: 2 . Stock:  50.0 , needed:  60.0 , new stock: 0
Lot has enough material 2 left. Stock:  2400.0 , needed:  10.0 , new stock:  2390.0
Panel_id: 330
Lot has enough material 4 left. Stock:  1836.0 , needed:  2.0 , new stock:  18

In [6]:
query = """
SELECT 
pic.lot_id,
sum(pic.quantity),
min(pl.updated_at) as moved_to_prod
FROM production.inventory_changelog pic
JOIN production.lot pl
ON pic.lot_id = pl.id
WHERE
    bin_id = 1
GROUP BY
    pic.lot_id
"""
available_lot = read_query_as_df(query)
display(available_lot)

,lot_id,sum,moved_to_prod
0,25,3600.0,2022-04-26 23:15:04.750953+00
1,21,1800.0,2022-04-26 23:13:41.130695+00
2,22,320.0,2022-04-26 23:13:48.285447+00
3,26,60000.0,2022-04-26 23:15:13.721354+00
4,27,130.0,2022-04-26 23:16:18.581899+00
5,23,1000.0,2022-04-26 23:13:55.402794+00
6,24,1000.0,2022-04-26 23:14:19.83506+00


In [39]:
conn = psycopg2.connect(user="jzztvyjdirgomm", password="974386311e9bf8265574baead65862ee677601c0f8e05bc954785e899d86dfaa", host="ec2-34-247-151-118.eu-west-1.compute.amazonaws.com",port="5432",database="djaki03gmcu3o")
cur = conn.cursor()
query = """INSERT INTO production.inventory_changelog (quantity, change_type_id, lot_id) VALUES (%s, 1, %s)
"""
cur.execute(query, [str(-10), str(available_lot["lot_id"].iloc[0])] )
conn.commit()
conn.close()

In [70]:
material_req['id'].unique()

array([284, 283, 282, 281, 280, 279, 278, 277, 276, 275, 274, 273, 272,
       271, 270, 269, 268, 267, 266, 265, 264, 263, 262, 261, 260, 259,
       258, 257, 256, 255, 254, 253, 252, 251, 250, 249, 248, 247, 246,
       245, 244, 243, 242, 241, 240, 239, 238, 237, 236, 235, 234, 233,
       232, 231, 230, 229, 228, 227, 226, 225, 224, 223, 222, 221, 220,
       219, 218, 217, 216, 215, 214, 213, 212, 211, 210, 209, 208, 207,
       206, 205, 204, 203, 202, 201, 200, 199, 198, 197, 196, 195, 194,
       193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181,
         1,  34,  33,  32,  31,  30,  29,  27,  26,  25,  24,  35,  38,
        23,  22,  21,  20,  19,  18,  17,  37,  16,  15,  14,  13,  12,
        11,  10,   9,   8,   7,   6,   5,   4,   3,   2,  36,  28,  39,
       300, 299, 298, 295, 296, 297, 294, 293, 292, 291, 290, 289, 288,
       287, 286, 285, 180, 179, 178, 177, 176, 175, 174, 173, 172, 171,
       170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 15

In [19]:
conn.commit()
conn.close()

In [8]:
panels = material_req['id'].unique()

In [7]:
material_req.sort_values(by="created_at", inplace = True)
#display(material_req)
panels = material_req['id'].unique()
print(panels[225:226])

[114]


In [40]:
len(panels)

253

In [43]:
#EL test part
import psycopg2
import psycopg2.extras
import pandas as pd

query = """
SELECT
psp.id, psp.created_at,
ppmr.material_id, 
ppmr.amount
FROM production.solar_panel psp
Full JOIN production.panel_material_req ppmr
ON psp.panel_type_id = ppmr.panel_type_id
JOIN production.material pm
ON pm.id = ppmr.material_id
JOIN production.material_type pmt
ON pmt.id = pm.material_type_id
WHERE pmt.type in ('Tabbing Ribbon', 'Solar Cell')
"""
material_req = read_query_as_df(query)
display(material_req)

,id,created_at,material_id,amount
0,284,2020-12-10 10:32:15.290776,5,0.3
1,284,2020-12-10 10:32:15.290776,2,60.0
2,283,2020-12-10 10:32:15.290777,5,0.3
3,283,2020-12-10 10:32:15.290777,2,60.0
4,282,2020-12-10 10:32:15.290778,5,0.3
...,...,...,...,...
501,118,2022-04-12 05:59:48.766311,12,60.0
502,116,2022-04-12 06:08:58.958990,5,0.3
503,116,2022-04-12 06:08:58.958990,12,60.0
504,115,2022-04-12 11:48:26.194608,5,0.3


In [ ]:
#EL test, part 2
material_req.sort_values(by="created_at", inplace = True)
#display(material_req)
panels = material_req['id'].unique()
#panels = [284]

for panel in panels[150:]:
    conn = psycopg2.connect(user="jzztvyjdirgomm", password="974386311e9bf8265574baead65862ee677601c0f8e05bc954785e899d86dfaa", host="ec2-34-247-151-118.eu-west-1.compute.amazonaws.com",port="5432",database="djaki03gmcu3o")
    cur = conn.cursor()
    material = material_req[material_req["id"] == panel]
    #display(material)
    print("Panel_id:", panel)
    for index, line in material.iterrows():
        query = """
        SELECT 
        pic.lot_id, 
        sum(pic.quantity),
        min(pl.updated_at) as moved_to_prod
        FROM production.inventory_changelog pic
        JOIN production.lot pl
        ON pic.lot_id = pl.id
        JOIN finance.purchase_order_line ppol
        ON ppol.id = pl.purchase_order_line_id
        WHERE
            bin_id = 2 AND
            ppol.material_id = %s
        GROUP BY
            pic.lot_id
        """%line["material_id"]
        available_lot = read_query_as_df(query)
        
        if available_lot.empty:
            print('Material', line["material_id"],'never added to production')
            break
        if available_lot["sum"].sum() == 0:
            print("No more material left")
            break
        #display(available_lot)
        for index2, row in available_lot.iterrows():
            if round(row["sum"],4) == 0: #if lot empty, go to the next lot
                pass
            elif row["sum"] >= line["amount"]:
                print("Lot has enough material",line["material_id"] ,"left. Stock: ", round(row["sum"],4), ", needed: ", round(line["amount"],4), ", new stock: ", round(row["sum"] - line["amount"],4))
                query = """INSERT INTO production.inventory_changelog (quantity, change_type_id, lot_id, created_at, panel_id) 
                VALUES (%s, 10, %s, %s, %s)
                """
                cur.execute(query, [str(round(-line["amount"],4)), str(row["lot_id"]), line["created_at"], str(panel)])
                line["amount"] = 0
            else:
                print("Multiple lots needed for material:",round(line["material_id"],4) ,". Stock: ", round(row["sum"],4), ", needed: ", round(line["amount"],4), ", new stock: 0")
                #Reduce first lot to 0
                query = """INSERT INTO production.inventory_changelog (quantity, change_type_id, lot_id, created_at, panel_id)
                VALUES (%s, 10, %s, %s, %s)
                """
                cur.execute(query, [str(round(-row["sum"],4)), str(row["lot_id"]), line["created_at"], str(panel)])
                line["amount"] = round(line["amount"]- row["sum"],4) #Reduce lot by remaining stock
                
            if line["amount"] == 0:
                break
        if line["amount"] > 0:
            print('Not enough Material', line["material_id"])
            break
    conn.commit()
    conn.close()